In [2]:
# load data files
# https://www.irs.gov/statistics/soi-tax-stats-migration-data

from io import BytesIO
import re
import requests
import requests_cache
import pandas as pd

requests_cache.install_cache()

data_keys = ['1112', '1213', '1314', '1415', '1516']

pd_options = {}

def get_data_url(data_key):
    return f'https://www.irs.gov/pub/irs-soi/{data_key}inmigall.csv'

dfs = []

for key in data_keys:
    url = get_data_url(key)
    r = requests.get(url)
    
    with BytesIO(r.content) as fh:
        df = pd.read_csv(fh, **pd_options)
        df['timeframe'] = key
        dfs.append(df)
        
mig_df = pd.concat(dfs).set_index(['statefips', 'state', 'state_name', 'agi_stub', 'timeframe']).sort_index()

mig_df.index.set_levels([
    "Total",
    (1,1e4),
    (1e4, 2.5e4),
    (2.5e4, 5e4),
    (5e4, 7.5e4),
    (7.5e4, 1e5),
    (1e5, 2e5),
    (2e5,)
#     "$1 under $10,000",
#     "$10,000 under $25,000",
#     "$25,000 under $50,000",
#     "$50,000 under $75,000",
#     "$75,000 under $100,000",
#     "$100,000 under $200,000",
#     "$200,000 or more",
], level = 'agi_stub', inplace=True)

display(mig_df.head(n=10))

total_n1_0  total_n2_0  \
statefips state state_name agi_stub     timeframe                           
1         AL    ALABAMA    Total        1112          1679103     3807951   
                                        1213          1673858     3788019   
                                        1314          1663655     3748068   
                                        1415          1660788     3712732   
                                        1516          1668163     3703054   
                           (1, 10000.0) 1112           160174      266204   
                                        1213           151289      249878   
                                        1314           143816      237017   
                                        1415           137619      225593   
                                        1516           133267      215805   

                                                   total_y1_agi_0  \
statefips state state_name agi_stub     timeframe                   
1         AL    ALABAMA    Total        1112             90691917   
                                        1213             92432250   
                                        1314             97120991   
                                        1415             95926666   
                                        1516            100346717   
                           (1, 10000.0) 1112              1819941   
                                        1213              1707255   
                                        1314              1677861   
                                        1415              1608564   
                                        1516              1580537   

                                                   total_y2_agi_0  total_n1_1  \
statefips state state_name agi_stub     timeframe                               
1         AL    ALABAMA    Total        1112             93398378      172506   
                                        1213             98657104      172504   
                                        1314             97625193      171347   
                                        1415            101230618      172947   
                                        1516            105018157      173605   
                           (1, 10000.0) 1112              1017090       34539   
                                        1213               957335       31838   
                                        1314               907684       30561   
                                        1415               867298       29319   
                                        1516               834010       27743   

                                                   total_n2_1  total_y1_agi_1  \
statefips state state_name agi_stub     timeframe                               
1         AL    ALABAMA    Total        1112           364898         3140327   
                                        1213           363798         3236937   
                                        1314           353586         3327271   
                                        1415           349905         3417966   
                                        1516           343526         3538976   
                           (1, 10000.0) 1112            54396          325485   
                                        1213            49595          307675   
                                        1314            47026          291201   
                                        1415            44733          278283   
                                        1516            42043          262349   

                                                   total_y2_agi_1  total_n1_2  \
statefips state state_name agi_stub     timeframe                               
1         AL    ALABAMA    Total        1112              3636450      303586   
                                        1213              3789283      299910   
                           

In [3]:
# single line chart output

targets = ['outflow_n1_0', 'inflow_n1_0']

df = mig_df.query('state == "IL" and agi_stub != "Total"')[targets]

df['net_flow'] = df.inflow_n1_0 - df.outflow_n1_0

# df.pivot(index='timeframe', columns='agi_stub', values='net_flow')
df = df.reset_index().pivot(index='timeframe', columns='agi_stub', values='net_flow')

df = df.rename(index=lambda x: f'1/1/20{x[2:]}')

df.index.names=['date']

df.columns = [
    "$1 under $10,000",
    "$10,000 under $25,000",
    "$25,000 under $50,000",
    "$50,000 under $75,000",
    "$75,000 under $100,000",
    "$100,000 under $200,000",
    "$200,000 or more",
]

In [4]:
# small mults output

targets = ['outflow_n1_0', 'inflow_n1_0']

df = mig_df.query('state == "IL" and agi_stub != "Total"')[targets]

df['net_flow'] = df.inflow_n1_0 - df.outflow_n1_0

df = df.reset_index().rename(columns={'timeframe': 'date', 'agi_stub': 'group'})[['group', 'date', 'net_flow']]

df.set_index(['group', 'date'], inplace=True)

df.index.set_levels([
    "$1 under $10,000",
    "$10,000 under $25,000",
    "$25,000 under $50,000",
    "$50,000 under $75,000",
    "$75,000 under $100,000",
    "$100,000 under $200,000",
    "$200,000 or more",
], level = 'group', inplace=True)

df.index.set_levels([
    '1/1/2012',
    '1/1/2013',
    '1/1/2014',
    '1/1/2015',
    '1/1/2016'
], level='date', inplace=True)

df.to_csv('sample.csv')

In [5]:
# single line chart output by state

targets = ['outflow_n1_0', 'inflow_n1_0']

df = mig_df.query('agi_stub == "Total"')[targets]

df['net_flow'] = df.inflow_n1_0 - df.outflow_n1_0

df = df.reset_index().pivot(index='timeframe', columns='state', values='net_flow')

df = df.rename(index=lambda x: f'1/1/20{x[2:]}')

df.index.names=['date']

df.to_csv('sample.csv')

In [14]:
# small mults output by income, many states

targets = ['outflow_n1_0', 'inflow_n1_0']

df = mig_df.query('agi_stub != "Total"')[targets]

df['net_flow'] = df.inflow_n1_0 - df.outflow_n1_0

df = df.reset_index().rename(columns={'timeframe': 'date', 'agi_stub': 'group'})

pivot = pd.pivot_table(df, values='net_flow', columns='state', index=['group', 'date'])

pivot.index.set_levels([
    "$1 under $10,000",
    "$10,000 under $25,000",
    "$25,000 under $50,000",
    "$50,000 under $75,000",
    "$75,000 under $100,000",
    "$100,000 under $200,000",
    "$200,000 or more",
], level = 'group', inplace=True)

pivot.index.set_levels([
    '1/1/2012',
    '1/1/2013',
    '1/1/2014',
    '1/1/2015',
    '1/1/2016'
], level='date', inplace=True)

# pivot.to_csv('sample.csv')

In [16]:
# small mults output by income, many states

targets = ['outflow_n1_0', 'inflow_n1_0']

df = mig_df.query('agi_stub != "Total"')[targets]

df['net_flow'] = df.inflow_n1_0 - df.outflow_n1_0

df = df.reset_index().rename(columns={'timeframe': 'date', 'agi_stub': 'group'})

pivot = pd.pivot_table(df, values=targets, index=['state', 'group', 'date'])

pivot.index.set_levels([
    "$1 under $10,000",
    "$10,000 under $25,000",
    "$25,000 under $50,000",
    "$50,000 under $75,000",
    "$75,000 under $100,000",
    "$100,000 under $200,000",
    "$200,000 or more",
], level = 'group', inplace=True)

pivot.index.set_levels([
    '1/1/2012',
    '1/1/2013',
    '1/1/2014',
    '1/1/2015',
    '1/1/2016'
], level='date', inplace=True)

# pivot.to_csv('sample.csv')
# pivot['IL']
pivot.query('state == "IL"')

inflow_n1_0  outflow_n1_0
state group                   date                               
IL    $1 under $10,000        1/1/2012        16358         17985
                              1/1/2013        14743         17328
                              1/1/2014        12617         16250
                              1/1/2015        10641         12971
                              1/1/2016        11161         14358
      $10,000 under $25,000   1/1/2012        29942         35656
                              1/1/2013        28665         35960
                              1/1/2014        25091         34028
                              1/1/2015        20936         27578
                              1/1/2016        24369         33150
      $25,000 under $50,000   1/1/2012        25952         30900
                              1/1/2013        26098         32177
                              1/1/2014        23701         31284
                              1/1/2015        17809         23041
                              1/1/2016        23989         31675
      $50,000 under $75,000   1/1/2012        12860         16169
                              1/1/2013        13674         18084
                              1/1/2014        12446         17944
                              1/1/2015         9188         11941
                              1/1/2016        13314         18982
      $75,000 under $100,000  1/1/2012         6835          9325
                              1/1/2013         7673         10702
                              1/1/2014         6924         10750
                              1/1/2015         4855          7122
                              1/1/2016         7469         11871
      $100,000 under $200,000 1/1/2012         8711         12896
                              1/1/2013         9490         15016
                              1/1/2014         8983         15536
                              1/1/2015         6168         10154
                              1/1/2016        10118         18351
      $200,000 or more        1/1/2012         3516          5920
                              1/1/2013         4132          6916
                              1/1/2014         3891          6998
                              1/1/2015         2887          5478
                              1/1/2016         4737          9273